In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import tensorflowjs as tfjs

import numpy as np
import pandas as pd

In [2]:
# Constantes
HEIGHT = 512
WIDTH = 512
CHANNELS = 3
NUM_EPOCHS = 8
NUM_CLASS = 2
tamaño_lote = 32

In [3]:
# Cargar el archivo CSV
df = pd.read_csv('./data/image_data.csv')

# Obtener las rutas de las imágenes y las etiquetas correspondientes
image_paths = ['./data/cleaned_images/' + name for name in df['name']]
labels = df['category']

In [4]:
df

,name,category
0,can_aceptable_12.jpg,aceptable
1,plastic_no_aceptable_11.jpg,no
2,plastic_no_aceptable_39.jpg,no
3,plastic_aceptable_43.jpg,aceptable
4,can_no_aceptable_48.jpg,no
...,...,...
334,plastic_no_aceptable_20.jpg,no
335,plastic_aceptable_7.jpg,aceptable
336,plastic_no_aceptable_34.jpg,no
337,can_aceptable_23.jpg,aceptable


In [5]:
# Crear listas vacías para almacenar las imágenes y las etiquetas preprocesadas
processed_images = []
processed_labels = []

# Recorrer las rutas de las imágenes
for image_path, label in zip(image_paths, labels):
    # Cargar la imagen
    image = load_img(image_path, target_size=(HEIGHT, WIDTH))
    # Convertir la imagen a un arreglo NumPy
    image_array = img_to_array(image)
    # Normalizar los valores de los píxeles entre 0 y 1
    image_array /= 255.0
    # Agregar la imagen preprocesada y la etiqueta a las listas correspondientes
    processed_images.append(image_array)
    processed_labels.append(label)

# Convertir las listas a arreglos NumPy
processed_images = np.array(processed_images)
processed_labels = np.array(processed_labels)

In [6]:
# Definir el modelo CNN
def crear_modelo_cnn():
    """
    altura: Representa la altura de las imágenes que se utilizarán como entrada para el modelo CNN. 
    Debes definir este valor según las dimensiones de tus imágenes.

    anchura: Representa la anchura de las imágenes que se utilizarán como entrada para el modelo CNN. 
    Debes definir este valor según las dimensiones de tus imágenes.

    canales: Representa el número de canales de las imágenes que se utilizarán como entrada para el modelo CNN. 
    Si las imágenes son en escala de grises, el valor típico es 1. Si las imágenes son en color (RGB), el valor típico es 3 (uno para cada canal de rojo, verde y azul).

    num_clases: Representa el número de clases o categorías diferentes a las que deseas clasificar tus imágenes. 
    Debes definir este valor según la cantidad de clases que tienes en tu conjunto de datos.
    """
    modelo = tf.keras.Sequential()
    
    # Capa convolucional 1
    modelo.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(HEIGHT, WIDTH, CHANNELS)))
    modelo.add(layers.MaxPooling2D((2, 2)))
    
    # Capa convolucional 2
    modelo.add(layers.Conv2D(64, (3, 3), activation='relu'))
    modelo.add(layers.MaxPooling2D((2, 2)))
    
    # Capa convolucional 3
    modelo.add(layers.Conv2D(128, (3, 3), activation='relu'))
    modelo.add(layers.MaxPooling2D((2, 2)))
    
    # Capa completamente conectada
    modelo.add(layers.Flatten())
    modelo.add(layers.Dense(128, activation='relu'))
    modelo.add(layers.Dense(NUM_CLASS, activation='softmax'))
    
    return modelo

In [7]:
# Crear el modelo
modelo = crear_modelo_cnn()

# Compilar el modelo


modelo.compile(optimizer='adam',
               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

In [8]:
# Dividir los datos en conjuntos de entrenamiento y prueba

datos_entrenamiento, datos_prueba, etiquetas_entrenamiento, etiquetas_prueba = train_test_split(
    processed_images, processed_labels, test_size=0.2, random_state=42)

In [9]:
etiquetas_prueba

array(['no', 'no', 'no', 'aceptable', 'aceptable', 'aceptable', 'no',
       'no', 'no', 'aceptable', 'no', 'aceptable', 'no', 'no',
       'aceptable', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'aceptable', 'aceptable', 'no', 'aceptable', 'no', 'no', 'no',
       'no', 'no', 'no', 'aceptable', 'no', 'no', 'aceptable', 'no',
       'aceptable', 'no', 'aceptable', 'aceptable', 'no', 'aceptable',
       'aceptable', 'no', 'no', 'aceptable', 'no', 'aceptable', 'no',
       'aceptable', 'no', 'no', 'no', 'no', 'aceptable', 'no', 'no',
       'aceptable', 'no', 'no', 'aceptable', 'no', 'no', 'no',
       'aceptable', 'no'], dtype='<U9')

In [10]:
etiquetas_entrenamiento = np.where(etiquetas_entrenamiento == 'aceptable', 1, 0)
etiquetas_prueba = np.where(etiquetas_prueba == 'no', 1, 0)

In [11]:
etiquetas_prueba

array([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1])

In [12]:
# Entrenar el modelo
modelo.fit(datos_entrenamiento, etiquetas_entrenamiento, epochs=NUM_EPOCHS, batch_size=tamaño_lote)

Epoch 1/10


/opt/homebrew/Caskroom/miniconda/base/envs/dataScience/lib/python3.10/site-packages/keras/src/backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


5/5 [==============================] - 16s 3s/step - loss: 8.7159 - accuracy: 0.5018
Epoch 2/10
5/5 [==============================] - 15s 3s/step - loss: 0.6444 - accuracy: 0.6753
Epoch 3/10
5/5 [==============================] - 16s 3s/step - loss: 0.5690 - accuracy: 0.7306
Epoch 4/10
5/5 [==============================] - 16s 3s/step - loss: 0.5029 - accuracy: 0.6790
Epoch 5/10
5/5 [==============================] - 15s 3s/step - loss: 0.3581 - accuracy: 0.8672
Epoch 6/10
5/5 [==============================] - 15s 3s/step - loss: 0.4429 - accuracy: 0.7749
Epoch 7/10
5/5 [==============================] - 15s 3s/step - loss: 0.2624 - accuracy: 0.8967
Epoch 8/10
5/5 [==============================] - 15s 3s/step - loss: 0.2373 - accuracy: 0.9262
Epoch 9/10
5/5 [==============================] - 15s 3s/step - loss: 0.1663 - accuracy: 0.9557
Epoch 10/10
5/5 [==============================] - 15s 3s/step - loss: 0.1217 - accuracy: 0.9631


In [13]:
# Evaluar el modelo
pérdida, precisión = modelo.evaluate(datos_prueba, etiquetas_prueba)
print('Pérdida:', pérdida)
print('Precisión:', precisión)

# Utilizar el modelo para hacer predicciones
#predicciones = modelo.predict(datos_nuevas_imagenes)
#etiqueta_predicha = etiquetas_clases[np.argmax(predicciones)]
#print('Etiqueta predicha:', etiqueta_predicha)

3/3 [==============================] - 1s 300ms/step - loss: 6.4674 - accuracy: 0.2647
Pérdida: 6.467443943023682
Precisión: 0.2647058963775635


In [14]:
print("etiquetas_entrenamiento data type:", type(etiquetas_entrenamiento))
print("etiquetas_entrenamiento shape:", etiquetas_entrenamiento.shape)

print("etiquetas_prueba data type:", type(etiquetas_prueba))
print("etiquetas_prueba shape:", etiquetas_prueba.shape)


etiquetas_entrenamiento data type: <class 'numpy.ndarray'>
etiquetas_entrenamiento shape: (271,)
etiquetas_prueba data type: <class 'numpy.ndarray'>
etiquetas_prueba shape: (68,)


In [15]:
# Generate predictions on the training data
y_pred_entrenamiento = modelo.predict(datos_entrenamiento)

# Calculate predicted probabilities for the positive class
y_pred_entrenamiento_positive_class = y_pred_entrenamiento[:, 1]

# Convert predicted probabilities to binary class labels
y_pred_entrenamiento_binary = (y_pred_entrenamiento_positive_class > 0.5).astype(int)

# Calculate F1-Score for training data
f1_train = f1_score(etiquetas_entrenamiento, y_pred_entrenamiento_binary)

print('F1-Score on Training Data:', f1_train)

9/9 [==============================] - 4s 464ms/step
F1-Score on Training Data: 0.9772727272727273


Exporting the model

In [16]:
modelo.save('cnn_model.h5')
#tfjs.converters.save_keras_model(modelo, './model')

/opt/homebrew/Caskroom/miniconda/base/envs/dataScience/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
